In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize WebDriver (change the path to your WebDriver)
driver = webdriver.Chrome()

# Search query
query = {
    "Love":"Christian Gospel Preaching on Love", 
    "Faith": "Christian Gospel preaching on Faith",
    "Hope": "Christian Gospel preaching on Hope",
    "Charity": "Christian Gospel preaching on Charity",
    "Prayer": "Christian Gospel preaching on Praying",
    "Praises": "Christian Gospel preaching on Praises",
    "Worship": "Christian Gospel preaching on Worship"
}
# https://www.youtube.com/results?search_query=Joshua+Selman+Sermons&sp=EgQIBRAB

all_links = {}
for k, v in query.items():
    url = f"https://www.youtube.com/results?search_query={v.replace(' ', '+')}&sp=EgQIBRAB"

# Open the YouTube Shorts search page
    driver.get(url)

    # WebDriverWait(driver, 10).until(
    #     EC.presence_of_element_located((By.ID, "filter-menu"))
    # )
    # time.sleep(5)
    # Click the filter button and select "This Month" (or similar)
    # driver.find_element(By.XPATH, "//*[@id='filter-button']/ytd-button-renderer/yt-button-shape/button").click()
    # time.sleep(1)
    # driver.find_element(By.XPATH, "//span[text()='This year'").click()
    # Wait for the page to load
    time.sleep(5)

    # Scroll to load more results
    for _ in range(50):  # Adjust the range to scroll more
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)

    # Find all video links
    videos = driver.find_elements(By.XPATH, "//a[@id='thumbnail']")

    # Extract and print video URLs
    links = []
    for video in videos:
        if href := video.get_attribute("href"):
            links.append(href)
            
    all_links[k] = links
    # Close the browser
driver.quit()

# Print the collected Shorts links
print(f"Collected all Links: {len(all_links.values())}")

Collected all Links: 7


In [3]:
universal = all_links.copy()

In [6]:
def extract_video_id(url):
    # Example URL formats:
    # https://www.youtube.com/watch?v=VIDEO_ID
    # https://youtu.be/VIDEO_ID
    # https://www.youtube.com/shorts/VIDEO_ID
    if "youtube.com/watch?v=" in url:
        return url.split("v=")[1].split("&")[0]
    elif "youtu.be/" in url:
        return url.split("youtu.be/")[1].split("?")[0]
    elif "youtube.com/shorts/" in url:
        return url.split("shorts/")[1].split("?")[0]
    else:
        return None


In [7]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import NoTranscriptFound, VideoUnavailable

def get_transcript(video_url):
    try:
        # Extract video ID from URL
        video_id = extract_video_id(video_url)
        # Fetch transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        # Combine transcript into a single string
        return " ".join([entry['text'] for entry in transcript])
    except NoTranscriptFound:
        print(f"No transcript found for video: {video_url}")
        return None
    except VideoUnavailable:
        print(f"Video unavailable: {video_url}")
        return None

# Fetch and save transcripts
# transcripts = []
# i = 0
# for link in links:
#     try:
#         if transcript := get_transcript(link):
#             transcript_dict = {"label": i, "transcript": transcript, "source": link}
#             transcripts.append(transcript_dict.copy())
#             print(f"Added transcript for: {link} ; index: {i}" )
#             i += 1
#         else:
#             print(f"Skipping video {link} due to an error or missing transcript.")
#     except Exception as e:
#         print(e)
#         continue

In [8]:
from pytubefix import YouTube

def get_video_data(minister, video_url):
    yt = YouTube(video_url)
    print(yt.title)
    return {
        "minister": minister,
        "channel_name": yt.author,
        "title": yt.title,
        "description": yt.description,
        "views": yt.views,
        "length": yt.length,
        "transcript": None,
        "source": video_url,
    }
# len(links)  

# # Example usage



In [9]:
import pandas as pd
import os, json

def save_json(file, data):
    file_path = f"{file}.json"
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            existing_data = json.load(f)
        existing_data.extend(data)
        with open(file_path, "w") as f:
            f.write(json.dumps(existing_data, indent=2))
    else:
        try:
            with open(file_path, "w") as f:
                f.write(json.dumps(data, indent=2))
        except Exception as e:
            print(e)



In [8]:
all_links.items()

dict_items([('Love', ['https://www.googleadservices.com/pagead/aclk?sa=L&ai=CgghZlqR9Z4u6D_yQ59MPyrjwUZjyx518qJng8N8TZBABIABgt9SDhZQxggEXY2EtcHViLTYyMTk4MTE3NDcwNDkzNzGgAc6WyeECqAMEqgSnAk_QGS-lMF4L9f8C2GFbK1rJ9_mrtDzU2umRyscDl0JlzJHqj-P2H7gUX7l2Bb3qhFXH1BW5wYh16mZ7_HLLLrojjTJYNiqXMyV23P9Ft5cdcnfK7uWSWo74yyuAtsWUhXj1YvWJYqKkyNSrBY_IHCHonyLT82PDkejXn1hbdnyFEbLz_G6HIXHT767ZXJ-VWzkaoJFwz9LwMK09ff22qqZ_MVbdpuVKZKGciO8gFZEYUyMrR4xVWUfM_p-k829hvsrYKOrkmbH9MNTQrg4kccnaozhT8CnW_3aI1rYYf7tLtr174EaAcrWI-a1rdgOm8akyrxBwm2UzE72bHmb1Il-2zDVeN_9q3bfnMc7yZ5fdG2JaZxbntpkqQmFgHREDiLhthTWNlq-SBQ0IEjCjgZXU5dWP048BkgUICBN49urVngGgBlWAB5rptp4BkAcEqAf5sbECqAeAurECqAfywLECqAe4xLECqAfnnbECqAfonbECqAfqt7ECqAentbECqAfvtbECqAfwtbECqAeCqrECqAeECKgHqNIbqAe2B6gHsdwbqAeRn7ECqAewm7ECqAeusbECqAfmvLECqAemqrECqAfKurECqAfswLECqAf9srECqAf3uLECqAf4uLECqAelzLECqAeBxhuoB_yvsQKoB9XOG6gHq8UbqAeVCKgHnNwbqAe3obECqAfetbECkggLajZZLXJscUZRS02oCAHSCCgIgEEQARheMgKCAjoLgEKAwICAgKCAgAJI2aDSNVAUWPjLlsXO5IoDmgk1aHR0cHM6Ly93d3cuZ29kbGlmZS5

In [10]:
from typing import DefaultDict

# Fetch and save transcripts
i = 0
for k, v in all_links.items():
    data = []
    for link in v[:]:
        try:
            video_data = get_video_data(video_url=link,minister=k)
            if transcript := get_transcript(video_url=link):
                video_data["transcript"] = transcript
                # print(video_data)
                print(f"Added transcript for: {k} ; index: {i}" )
                data.append(video_data.copy())
                i += 1
            else:
                print(f"Skipping video {link} due to an error or missing transcript.")
        except Exception as e:
            print(e)
            continue
    save_json(data= data, file="virtues")

<urlopen error [Errno -3] Temporary failure in name resolution>
<urlopen error [Errno -3] Temporary failure in name resolution>


KeyboardInterrupt: 

In [32]:
print(data[0])

{'minister': 'Pastor Enoch Adeboye', 'channel_name': 'Divine Encounter 🕊️', 'title': 'PASTOR E.A ADEBOYE SERMON | OPEN MY EYES', 'description': "Welcome to Divine Encounter \n\nYou are Watching PASTOR E.A ADEBOYE SERMON | OPEN MY EYES\n\nJoin us for our services, and tune in for daily broadcasts of prayer, praise, worship, and sermons. #DivineEncounter #RCCGSpecialService #RCCG\n\nWe are Redeemites!\n\nIf you are blessed by our content, please like, share, and leave a comment below. We're dedicated to providing resources to equip young aspiring pastors, music ministers, and church group leaders for success in ministry.\n\nNew videos are uploaded daily, featuring praise and worship of the one true God and our Savior, Jesus Christ, who died for our sins so that we may be saved through Him.\n\nWe bring you the best of worship and sermons as we gather together for Sunday worship.\n\nWorshiping Jesus Christ is the single most important thing we do", 'views': 36998, 'length': 2461, 'transcri